In [238]:
import pandas as pd
import numpy as np
import os

from scipy.stats import combine_pvalues

In [239]:
def fisher_method_threshold(p_values, min_p_value = 1e-16):
    p_values = np.maximum(p_values, min_p_value)
    _, combined_p_value = combine_pvalues(p_values, method='fisher')
    return combined_p_value

In [240]:
#establish file paths to DEA results
proj_codes_df = pd.read_csv('proj_codes.csv')
proj_codes_df['matched_tissue'] = proj_codes_df['matched_tissue'].str.replace(' ','_')

base_dir = '/Users/samlokanc/projects/Lab/IEI/IEI 2024/Results'
file_paths_DEA = proj_codes_df.apply(
    lambda row: os.path.join(base_dir, row['proj'], 'DEA', f"subset_DEGs_{row['proj']}_{row['sample_abbr']}_GTEX_{row['matched_tissue']}.csv"),
    axis = 1
)

#load data from specified files into pandas dataframes
DEA_dfs = []

for i, file_path in enumerate(file_paths_DEA):
    df = pd.read_csv(file_path)
    df['proj'] = proj_codes_df.iloc[i]['proj']
    DEA_dfs.append(df)

In [241]:
#concatenate all DEA dataframes into single respective dataframe
DEA_data = pd.concat(DEA_dfs, ignore_index = True)

#count occurences of each gene
gene_counts = DEA_data['Gene.symbol'].value_counts().reset_index()
gene_counts.columns = ['Gene.symbol', 'count']

#calculate the fisher combined p-vals and log2fc for each gene
grouped_DEA_data = DEA_data.groupby('Gene.symbol').agg(
    combined_p_values = ('adj.P.Val', lambda p_values: fisher_method_threshold(p_values)),
    average_log2fc = ('logFC', 'mean')
).reset_index()

#aggregate project codes for each gene
project_codes = DEA_data.groupby('Gene.symbol')['proj'].apply(lambda x: list(x.unique())).reset_index()

#merge gene counts with grouped data
DEA_result_df = pd.merge(gene_counts, grouped_DEA_data, on='Gene.symbol')
DEA_result_df = pd.merge(DEA_result_df, project_codes, on='Gene.symbol')

#Filter to get significant top up/down regulated terms
top_upregulated = DEA_result_df[DEA_result_df['average_log2fc'] > 0].sort_values(by = 'average_log2fc', ascending = False)
top_downregulated = DEA_result_df[DEA_result_df['average_log2fc'] < 0].sort_values(by = 'average_log2fc', ascending = True)

In [242]:
#establish file paths to GO results
file_paths_GO = proj_codes_df.apply(
    lambda row: os.path.join(base_dir, row['proj'], 'GO', f"file_{row['proj']}_{row['sample_abbr']}_GTEX_{row['matched_tissue']}.csv"),
    axis = 1
)

#load data from specified files into pandas dataframes
GO_dfs =[]

for i, file_path in enumerate(file_paths_GO):
    df = pd.read_csv(file_path)
    df['proj'] = proj_codes_df.iloc[i]['proj']
    GO_dfs.append(df)

In [247]:
#concatenate all GO dataframes into single respective dataframe
GO_data = pd.concat(GO_dfs, ignore_index = True)

#remove non significant results
GO_data = GO_data[(GO_data['over_represented_pvalue'] < 0.05) | (GO_data['under_represented_pvalue'] < 0.05)]

#count occurences of each GO term
term_counts = GO_data['term'].value_counts().reset_index()
term_counts.columns = ['term', 'count']

#calculate the fisher combined p-vals and log2fc for each gene
grouped_GO_data = GO_data.groupby(['term','category']).agg(
    combined_overrepresented_pvalue = ('over_represented_pvalue', lambda p_values: fisher_method_threshold(p_values)),
    combined_underrepresented_pvalue = ('under_represented_pvalue', lambda p_values: fisher_method_threshold(p_values))
).reset_index()

#aggregate project codes for each gene
project_codes = GO_data.groupby('term')['proj'].apply(lambda x: list(x.unique())).reset_index()

#merge gene counts with grouped data
GO_result_df = pd.merge(term_counts, grouped_GO_data, on='term')
GO_result_df = pd.merge(GO_result_df, project_codes, on='term')

#Filter to get significant top up/down regulated genes
top_overrepresented = GO_result_df[GO_result_df['combined_overrepresented_pvalue'] < 0.05].sort_values(by = 'combined_overrepresented_pvalue', ascending = True)
top_underrepresented = GO_result_df[GO_result_df['combined_underrepresented_pvalue'] < 0.05].sort_values(by = 'combined_underrepresented_pvalue', ascending = True)

In [248]:
top_upregulated.to_csv('top_upregulated.csv', index = False)
top_downregulated.to_csv('top_downregulated.csv', index = False)

top_overrepresented.to_csv('top_overrepresented.csv', index = False)
top_underrepresented.to_csv('top_underrepresented.csv', index = False)

In [249]:
top_overrepresented

,term,count,category,combined_overrepresented_pvalue,combined_underrepresented_pvalue,proj
0,humoral immune response,23,GO:0006959,4.098413e-76,1.000000,"[TCGA-LAML, TCGA-STAD, TCGA-SKCM, TCGA-BRCA, T..."
3,extracellular region,20,GO:0005576,1.531687e-58,1.000000,"[TCGA-LAML, TCGA-STAD, TCGA-SKCM, TCGA-BRCA, T..."
7,complement activation,19,GO:0006956,2.030118e-49,1.000000,"[TCGA-LAML, TCGA-STAD, TCGA-BRCA, TCGA-BLCA, T..."
6,immune response,19,GO:0006955,5.013250e-47,1.000000,"[TCGA-LAML, TCGA-STAD, TCGA-SKCM, TCGA-BRCA, T..."
25,extracellular space,16,GO:0005615,1.588040e-46,1.000000,"[TCGA-LAML, TCGA-STAD, TCGA-BRCA, TCGA-BLCA, T..."
...,...,...,...,...,...,...
2146,cilium movement involved in cell motility,1,GO:0060294,4.988207e-02,0.992139,[TCGA-CESC]
2138,epithelial cilium movement involved in extrace...,1,GO:0003351,4.992200e-02,0.992130,[TCGA-CESC]
2145,extracellular transport,1,GO:0006858,4.992200e-02,0.992130,[TCGA-CESC]
2230,positive regulation of intrinsic apoptotic sig...,1,GO:1902255,4.999851e-02,1.000000,[TCGA-LAML]


In [267]:
UR = top_underrepresented[(top_underrepresented['count'] >= 14)]
UR = ', '.join(UR['category'].astype(str))
UR

'GO:0005622, GO:0043229, GO:0043231, GO:0031981, GO:0005737, GO:0005654, GO:0006974, GO:0005829, GO:0006996, GO:0005634, GO:0006281, GO:0033554, GO:0043232, GO:0043228, GO:0140535, GO:0140513, GO:0043226, GO:0003824'

In [270]:
OR = top_overrepresented[(top_overrepresented['count'] >= 14)]
OR = ', '.join(OR['category'].astype(str))
OR

'GO:0006959, GO:0005576, GO:0006956, GO:0006955, GO:0005615, GO:0002252, GO:0002682, GO:0009897, GO:0002250, GO:0002455, GO:0002684, GO:0071944, GO:0002376, GO:0006958, GO:0002253, GO:0009986, GO:0072562, GO:0060089, GO:0038023, GO:0048583, GO:0050776, GO:0050778, GO:0002920, GO:0051241, GO:0048584, GO:0002698, GO:0006952, GO:0065007, GO:0062023, GO:0005886, GO:0030312, GO:0031012, GO:0002449, GO:0030098, GO:0001775, GO:0019724, GO:0002460, GO:0002921, GO:1903131, GO:0098552, GO:0045321, GO:0016064, GO:0072378, GO:0072376, GO:0002443, GO:0070661'

In [269]:
len(top_overrepresented[(top_overrepresented['count'] >= 14)])


46